In [1]:
import os
import pickle
import numpy as np
import pandas as pd
import random as r
import seaborn as sns
import matplotlib.pyplot as plt

from time import time
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, GridSearchCV

from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from sklearn.ensemble import AdaBoostRegressor, ExtraTreesRegressor, GradientBoostingRegressor, RandomForestRegressor
from xgboost import XGBRegressor

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [2]:
class CFG(object):
    def __init__(self,
         seed: int = 42,
         n_splits: int = 5,
         show_info: bool = False,
         ):

        self.seed = seed
        self.n_splits = n_splits
        self.show_info = show_info
        self.tr_data = "../input/heart-disease-prediction-tfug-chd-oct-2022/train.csv"
        self.ts_data = "../input/heart-disease-prediction-tfug-chd-oct-2022/test.csv"
        self.ss_data = "../input/heart-disease-prediction-tfug-chd-oct-2022/sample_submission.csv"
        self.model_save_path = "models"
        if not os.path.exists(self.model_save_path): os.makedirs(self.model_save_path)

cfg = CFG(seed=42, show_info=True)

In [3]:
def breaker(num: int=50, char: str="*") -> None:
    print("\n" + num*char + "\n")
    

def print_scores(mae: float, mse: float, r2: np.ndarray) -> None:
    print(f"MAE : {mae:.5f}")
    print(f"MSE : {mse:.5f}")
    print(f"R2   {r2:.5f}")
    

def get_scores(y_true: np.ndarray, y_pred: np.ndarray) -> tuple:
    return mean_absolute_error(y_true, y_pred), mean_squared_error(y_true, y_pred), r2_score(y_true, y_pred)

In [4]:
class Model(object):
    def __init__(self, model_name: str, preprocessor, seed: int):
        self.model_name = model_name

        if self.model_name == "lr":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", LinearRegression()),
                ]
            )
        
        elif self.model_name == "knr":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", KNeighborsRegressor()),
                ]
            )
        
        elif self.model_name == "dtr":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", DecisionTreeRegressor(random_state=seed)),
                ]
            )

        elif self.model_name == "etr":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", ExtraTreeRegressor(random_state=seed)),
                ]
            )
        
        elif self.model_name == "rfr":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", RandomForestRegressor(random_state=seed)),
                ]
            )
        
        elif self.model_name == "gbr":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", GradientBoostingRegressor(random_state=seed)),
                ]
            )
        
        elif self.model_name == "abr":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", AdaBoostRegressor(random_state=seed)),
                ]
            )
        
        elif self.model_name == "etrs":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", ExtraTreesRegressor(random_state=seed)),
                ]
            )
        
        elif self.model_name == "svr":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", SVR()),
                ]
            )
        
        elif self.model_name == "xgbr":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", XGBRegressor(random_state=seed)),
                ]
            )

In [5]:
names = ["lr", "knr", "svr", "dtr", "etr", "abr", "gbr", "etrs", "rfr", "xgbr"]

tr_df = pd.read_csv(cfg.tr_data)
tr_df = tr_df.drop(columns=["index"])

X = tr_df.iloc[:, :-1].copy().values
y = tr_df.iloc[:, -1].copy().values

if cfg.show_info:
    breaker()
    for val in set(y):
        print(f"Class {val} count : {y[y == val].shape[0]}")

y = y.astype("float64")

features = [i for i in range(X.shape[1])]

feature_transformer = Pipeline(
    steps=[
        ("Standard_Scaler", StandardScaler())
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("features", feature_transformer, features),
    ]
)

best_mse: float = np.inf
for name in names:
    fold: int = 1
    if cfg.show_info: breaker()
    for tr_idx, va_idx in KFold(n_splits=cfg.n_splits, random_state=cfg.seed, shuffle=True).split(X):
        X_train, X_valid, y_train, y_valid = X[tr_idx], X[va_idx], y[tr_idx], y[va_idx]
        my_pipeline = Model(name, preprocessor, cfg.seed)
        my_pipeline.model.fit(X_train, y_train)

        y_pred = my_pipeline.model.predict(X_valid)
        if cfg.show_info:
            mae, mse, r2 = get_scores(y_valid, y_pred)
            print(f"{my_pipeline.model_name}, {fold}\n")
            print_scores(mae, mse, r2)
            print("")
        else:
            _, mse, _ = get_scores(y_valid, y_pred)

        if mse < best_mse:
            best_mse = mse
            model_fold_name = f"{name}_{fold}"

            with open(os.path.join(cfg.model_save_path, f"best_model.pkl"), "wb") as fp: pickle.dump(my_pipeline.model, fp)
        
        with open(os.path.join(cfg.model_save_path, f"{name}_{fold}.pkl"), "wb") as fp: pickle.dump(my_pipeline.model, fp)
        
        fold += 1
    

if cfg.show_info: 
    breaker()
    print(f"Best Model : {model_fold_name.split('_')[0]}, Best Fold : {model_fold_name.split('_')[1]}")

breaker()


**************************************************

Class 0 count : 386
Class 1 count : 454

**************************************************

lr, 1

MAE : 0.49943
MSE : 0.25738
R2   -0.03671

lr, 2

MAE : 0.49127
MSE : 0.24518
R2   -0.01302

lr, 3

MAE : 0.50495
MSE : 0.26244
R2   -0.06806

lr, 4

MAE : 0.49749
MSE : 0.25346
R2   -0.01743

lr, 5

MAE : 0.49479
MSE : 0.25124
R2   -0.01942


**************************************************

knr, 1

MAE : 0.48333
MSE : 0.28429
R2   -0.14509

knr, 2

MAE : 0.47262
MSE : 0.27548
R2   -0.13820

knr, 3

MAE : 0.49405
MSE : 0.30738
R2   -0.25098

knr, 4

MAE : 0.50357
MSE : 0.30214
R2   -0.21287

knr, 5

MAE : 0.50833
MSE : 0.30500
R2   -0.23754


**************************************************

svr, 1

MAE : 0.49997
MSE : 0.32079
R2   -0.29212

svr, 2

MAE : 0.49241
MSE : 0.30851
R2   -0.27470

svr, 3

MAE : 0.52236
MSE : 0.35292
R2   -0.43629

svr, 4

MAE : 0.49473
MSE : 0.33574
R2   -0.34772

svr, 5

MAE : 0.49848
MSE : 0.32493
R2 

In [6]:
ts_df = pd.read_csv(cfg.ts_data)
ts_df = ts_df.drop(columns=["index"])
X_test = ts_df.copy().values

model = pickle.load(open(os.path.join(cfg.model_save_path, f"best_model.pkl"), "rb"))
y_pred = model.predict(X_test)

ss_df = pd.read_csv(cfg.ss_data)
ss_df["target"] = y_pred
ss_df.to_csv("submission.csv", index=False)